In [1]:
import os
import pandas as pd
import h5py

%matplotlib inline

from IPython.display import display, HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
# Daysim data
trip = pd.read_csv(r'../../../outputs/daysim/_trip.tsv', sep='\t')
person = pd.read_csv(r'../../../outputs/daysim/_person.tsv', sep='\t')
hh = pd.read_csv(r'../../../outputs/daysim/_household.tsv', sep='\t')

In [3]:
def apply_labels(df,table_type):
    '''
    Replace values with human readable lablels.
    '''
    df_label = labels[labels['table'] == table_type]
    for field in df_label['field'].unique():
        newdf = df_label[df_label['field'] == field]
        local_series = pd.Series(newdf['text'].values, index=newdf['value'])
        df[field] = df[field].map(local_series)
    
    return df

In [4]:
# Add labels
labels = pd.read_csv(r'../../../scripts/summarize/inputs/calibration/variable_labels.csv')

trip = apply_labels(trip,'Trip')
person = apply_labels(person,'Person')
hh = apply_labels(hh,'Household')

In [5]:
special_taz = pd.read_csv(r'../../../scripts/summarize/inputs/special_needs_taz.csv')
rgc_taz = pd.read_csv(r'../../../scripts/summarize/inputs/rgc_taz.csv')

In [15]:
filename = r'../../../outputs/daysim/centers.xlsx'
if os.path.exists(filename):
    os.remove(filename)
writer = pd.ExcelWriter(filename)

# Mode Share to Centers by HH Location

## All Trips

In [16]:
trip_hh = pd.merge(trip,hh,on='hhno',how='left')
trip_hh = pd.merge(trip_hh, rgc_taz, left_on='hhtaz', right_on='taz', how='left')
trip_hh = trip_hh[trip_hh['mode']!='School Bus']

df = trip_hh[['trexpfac','geog_name','mode']].groupby(['geog_name','mode']).sum().reset_index()
tot_rgc = trip_hh[['trexpfac','geog_name']].groupby('geog_name').sum().reset_index()
df = pd.merge(df,tot_rgc,on='geog_name', suffixes=['_mode','_total'])
df['mode_share'] = (df['trexpfac_mode']/df['trexpfac_total'])

df = df.pivot(index='geog_name', columns='mode', values='mode_share')
# Trim the number of RGCs
rgc_list = ['Auburn','Bellevue','Bremerton','Everett','Issaquah','Lynnwood','Redmond Downtown',
       'Renton','Seattle Downtown','Seattle Northgate','Seattle University Community',
        'Seattle Uptown', 'Tacoma Downtown']


df.to_excel(writer,'HH in RGC - All Trips')
df.loc[rgc_list]

mode,Bike,HOV2,HOV3+,SOV,Transit,Walk
geog_name,,,,,,
Auburn,0.028282,0.213714,0.119797,0.311099,0.068837,0.258271
Bellevue,0.030374,0.151025,0.054560,0.239365,0.094506,0.430169
Bremerton,0.030780,0.158886,0.099620,0.252888,0.045339,0.412486
Everett,0.034945,0.147653,0.076774,0.253053,0.029858,0.457717
Issaquah,0.024430,0.157980,0.092834,0.495114,0.066775,0.162866
Lynnwood,0.029911,0.205891,0.126836,0.294715,0.054596,0.288051
Redmond Downtown,0.030135,0.157212,0.065385,0.320105,0.093029,0.334135
Renton,0.029879,0.165890,0.107278,0.294209,0.073848,0.328896
Seattle Downtown,0.050147,0.103374,0.055766,0.165351,0.137082,0.488275


## Commute Trips

In [17]:
commute_trips = trip_hh[trip_hh['dpurp'] == 'Work']

df = commute_trips[['trexpfac','geog_name','mode']].groupby(['geog_name','mode']).sum().reset_index()
tot_rgc = commute_trips[['trexpfac','geog_name']].groupby('geog_name').sum().reset_index()
df = pd.merge(df,tot_rgc,on='geog_name', suffixes=['_mode','_total'])
df['mode_share'] = (df['trexpfac_mode']/df['trexpfac_total'])

df = df.pivot(index='geog_name', columns='mode', values='mode_share')
# Trim the number of RGCs
rgc_list = ['Auburn','Bellevue','Bremerton','Everett','Issaquah','Lynnwood','Redmond Downtown',
       'Renton','Seattle Downtown','Seattle Northgate','Seattle University Community',
        'Seattle Uptown', 'Tacoma Downtown']
df = df.fillna(0)

df.to_excel(writer,'HH in RGC - Commute Trips')
df.loc[rgc_list]

mode,Bike,HOV2,HOV3+,SOV,Transit,Walk
geog_name,,,,,,
Auburn,0.051044,0.074246,0.053364,0.533643,0.097448,0.190255
Bellevue,0.037370,0.066570,0.022802,0.321890,0.119775,0.431594
Bremerton,0.048739,0.069954,0.028096,0.398509,0.061353,0.393349
Everett,0.037074,0.069904,0.027712,0.356010,0.065785,0.443515
Issaquah,0.017391,0.078261,0.017391,0.634783,0.113043,0.139130
Lynnwood,0.050000,0.091667,0.040625,0.440625,0.119792,0.257292
Redmond Downtown,0.043171,0.076622,0.026635,0.439914,0.149583,0.264075
Renton,0.040093,0.091142,0.034732,0.453846,0.101865,0.278322
Seattle Downtown,0.050401,0.055842,0.023932,0.233699,0.150080,0.486020


# Trips to Centers

## All Trips

In [18]:
trip_df = pd.merge(trip, rgc_taz, left_on='dtaz', right_on='taz', how='left')
trip_df = trip_df[trip_df['mode']!='School Bus']

df = trip_df[['trexpfac','geog_name','mode']].groupby(['geog_name','mode']).sum().reset_index()
tot_rgc = trip_df[['trexpfac','geog_name']].groupby('geog_name').sum().reset_index()
df = pd.merge(df,tot_rgc,on='geog_name', suffixes=['_mode','_total'])
df['mode_share'] = (df['trexpfac_mode']/df['trexpfac_total'])

df = df.pivot(index='geog_name', columns='mode', values='mode_share')
# Trim the number of RGCs
rgc_list = ['Auburn','Bellevue','Bremerton','Everett','Issaquah','Lynnwood','Redmond Downtown',
       'Renton','Seattle Downtown','Seattle Northgate','Seattle University Community',
        'Seattle Uptown', 'Tacoma Downtown']


df.to_excel(writer,'Trip to RGC - All Trips')
df.loc[rgc_list]

mode,Bike,HOV2,HOV3+,SOV,Transit,Walk
geog_name,,,,,,
Auburn,0.014651,0.217302,0.132509,0.460345,0.035972,0.139221
Bellevue,0.020710,0.138184,0.072815,0.279780,0.130609,0.357902
Bremerton,0.025652,0.171939,0.106825,0.339382,0.033400,0.322802
Everett,0.021917,0.175576,0.111464,0.323696,0.028213,0.339134
Issaquah,0.009260,0.195197,0.106362,0.553346,0.040759,0.095077
Lynnwood,0.019428,0.214920,0.135169,0.405961,0.034556,0.189965
Redmond Downtown,0.022419,0.190049,0.111052,0.337265,0.065434,0.273781
Renton,0.020141,0.192503,0.124697,0.396669,0.053118,0.212873
Seattle Downtown,0.030081,0.113147,0.065098,0.212002,0.212006,0.367664


## Commute Trips

In [19]:
commute_trips = trip[trip['dpurp'] == 'Work']
trip_df = pd.merge(commute_trips, rgc_taz, left_on='dtaz', right_on='taz', how='left')
trip_df = trip_df[trip_df['mode']!='School Bus']

df = trip_df[['trexpfac','geog_name','mode']].groupby(['geog_name','mode']).sum().reset_index()
tot_rgc = trip_df[['trexpfac','geog_name']].groupby('geog_name').sum().reset_index()
df = pd.merge(df,tot_rgc,on='geog_name', suffixes=['_mode','_total'])
df['mode_share'] = (df['trexpfac_mode']/df['trexpfac_total'])

df = df.pivot(index='geog_name', columns='mode', values='mode_share')
# Trim the number of RGCs
rgc_list = ['Auburn','Bellevue','Bremerton','Everett','Issaquah','Lynnwood','Redmond Downtown',
       'Renton','Seattle Downtown','Seattle Northgate','Seattle University Community',
        'Seattle Uptown', 'Tacoma Downtown']


df.to_excel(writer,'Trip to RGC - Commute Trips')
df.loc[rgc_list]

mode,Bike,HOV2,HOV3+,SOV,Transit,Walk
geog_name,,,,,,
Auburn,0.026354,0.087096,0.043870,0.662864,0.058814,0.121003
Bellevue,0.020326,0.076787,0.037290,0.381899,0.219608,0.264090
Bremerton,0.047028,0.076664,0.034250,0.523514,0.043656,0.274889
Everett,0.024320,0.081898,0.039411,0.485886,0.058953,0.309532
Issaquah,0.012775,0.085346,0.041224,0.699839,0.071927,0.088889
Lynnwood,0.029098,0.103227,0.047462,0.599986,0.056441,0.163786
Redmond Downtown,0.026955,0.087766,0.040037,0.508051,0.117165,0.220026
Renton,0.023113,0.101332,0.046153,0.596600,0.075380,0.157422
Seattle Downtown,0.032103,0.065885,0.031165,0.293904,0.282165,0.294773


In [20]:
writer.save()